In [1]:
import torch
import scipy.io
import matplotlib.pylab as plt
import os
import cv2
import tqdm
from sklearn.model_selection import train_test_split 
import glob
from cls_hrnet import HighResolutionNet, get_cls_net
import json
from torchvision import transforms
from PIL import Image
import numpy as np

In [2]:
transform = transforms.Compose([transforms.Resize((820, 550)), transforms.ToTensor()])

In [3]:
folder = 'main'

In [4]:
label_list = scipy.io.loadmat(folder + '/' + 'label_list.mat')

In [5]:
label_list['label_list'][0][:5]

array([array(['null'], dtype='<U4'), array(['accessories'], dtype='<U11'),
       array(['bag'], dtype='<U3'), array(['belt'], dtype='<U4'),
       array(['blazer'], dtype='<U6')], dtype=object)

In [6]:
output_folder = folder + '/annotations/pixel-level/'
input_folder = folder + '/photos/'

In [14]:
inputs = [transform(Image.open(x)).unsqueeze(0) for x in sorted(glob.glob(f'{input_folder}*.jpg'))[:10]] # 1004
labels = [scipy.io.loadmat(x)['groundtruth'] for x in sorted(glob.glob(f'{output_folder}*.mat'))[:10]] 

In [15]:
inputs[0].shape

torch.Size([1, 3, 820, 550])

In [16]:
inputs = torch.cat(inputs)

In [17]:
inputs.shape

torch.Size([10, 3, 820, 550])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, shuffle=True)

In [19]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, shuffle=True) 

In [20]:
with open('params.json', 'r') as file:
    params = json.load(file)
print(params)

{'MODEL': {'EXTRA': {'STAGE1': {'NUM_MODULES': 1, 'NUM_CHANNELS': [64], 'BLOCK': 'BOTTLENECK', 'NUM_BLOCKS': [4], 'NUM_BRANCHES': 1, 'FUSE_METHOD': 'SUM'}, 'STAGE2': {'NUM_MODULES': 1, 'NUM_CHANNELS': [48, 96], 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4], 'NUM_BRANCHES': 2, 'FUSE_METHOD': 'SUM'}, 'STAGE3': {'NUM_MODULES': 4, 'NUM_CHANNELS': [48, 96, 192], 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4, 4], 'NUM_BRANCHES': 3, 'FUSE_METHOD': 'SUM'}, 'STAGE4': {'NUM_MODULES': 3, 'NUM_CHANNELS': [48, 96, 192, 384], 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4, 4, 4], 'NUM_BRANCHES': 4, 'FUSE_METHOD': 'SUM'}}, 'PRETRAINED': 'hrnet_w18_small_model_v1.pth'}}


In [17]:
# model = HighResolutionNet(params)

In [21]:
model = get_cls_net(params)

In [23]:
model(torch.tensor(X_train))

/Users/vanna/miniconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


RuntimeError: The size of tensor a (205) must match the size of tensor b (206) at non-singleton dimension 2